# Summarize any length of text with Amazon Bedrock
In this notebook, we will set up a system to summarize text from one long source, or multiple documents.  This is guided by these three main goals:
  1) Accept any length of input documents, and any number of seperate documents.
  2) Allow guided summaries.  These are manual or automatically defined suggestions on what should be included in the summary.
          -manual guidance can be to look for particular trends or insights and is helpful for creating longer summaries.
          -automatic guidance allows the LLM to critique its own summary, and rebuild it based on what should have been included. (goal 3 below)
  3) Create an automated feedback loop, allowing the LLM to critically identify and correct weaknesses in the summary.

This notebook contains two main functions, one for summarizing single, long texts, and one for summarizing groups of documents.  These tasks are different enough that they require different approached, and therefore are two different functions.  Supporting both of these function, we create four basic helper functions: get_chunks() for breaking long documents down, get_prompt() for creating the many different prompts used, ask_claude for connecting to Bedrock and sending our prompt to the LLM, and finally ask_claude_threaded which allows us to make multiple calls to Claude at the same time..

  1) get_chunks - this takes a long text, and returns chunks of appropriate sizes for summarazation.  By default, this includes overlap between chunks.
  2) get_prompt - this helper function builds an appropriate prompt for Claude 2, based on options passed in.  In general, this includes prompts for creating summaries, as well as prompts for assessing summary quality.
  3) ask_claude - this helper function sends a prompt to Claude 2, and handles some errors that may pop up from the Bedrock endpoint.
  4) ask_claude_threaded - a wrapper for ask_claude that allows many requests to be sent at the same time.
  
This notebook follows this layout:

  1) Set up the environment.
  2) Define our four helper functions.
  3) Set up the two main summarizing functions, using the four helper functions from part (2).
  4) Explore using the main summarizing functions for maximum summary quality.
  
NOTE:  This notebook by default will load a cache of calls to Claude into memeory.  This is helpful when testing and itterating on multi-step prompts, but should be turned off if you would like to generate new versions of the same request.

## 1) Set up the environment
First, let's install some dependances:

Note: We install the Anthropic SDK to do local token counting, but do not actually use this in any way to call out to Anthropic.

In [2]:
#!pip install anthropic

In [132]:
from anthropic import Anthropic
client = Anthropic()
def count_tokens(text):
    return client.count_tokens(text)
#count_tokens('Hello world what is up?!')

6

In [29]:
#we'll use pickle to load text from our text cleaning notebook, but you can load any text into the full_text variable.  It should be stored as a single string.
import pickle, os, re, json
#we'll use time to track how long Bedrock takes to respond, which helps to estimate how long a job will take.
import time

#for ask_claude_threaded, import our threading libraries.
from queue import Queue
from threading import Thread

In [6]:
#set CACHE_RESPONCES to true to save all responses from Claude for reuse later.
#when true, and request to Claude that has been made before will be served from this cache, rather then sending a request to Bedrock.
CACHE_RESPONCES = False
if CACHE_RESPONCES: print ("WARNING: Claude Cache is enabled.  Responses may be stale, and this should be turned off in helper_functions during production use to prevent memory overflow.")

Next, let's set up the connection to Bedrock:
If needed, install at least 1.28.57 of Boto3 so that Bedrock is included.

In [18]:
#!pip install update boto3==1.28.57

In [ ]:
#for connecting with Bedrock, use Boto3
import boto3
from botocore.config import Config

#increase the standard time out limits in boto3, because Bedrock may take a while to respond to large requests.
my_config = Config(
    connect_timeout=60*3,
    read_timeout=60*3,
)

In [ ]:
bedrock = boto3.client(service_name='bedrock-runtime',config=my_config)
bedrock_service = boto3.client(service_name='bedrock',config=my_config)

In [133]:
#check that it's working:
models = bedrock_service.list_foundation_models()
if "anthropic.claude-v2" in str(models):
    pass#print("Claud-v2 found!")
else:
    print ("Error, no model found.")
max_token_count = 12000 #property of Claude 2

Claud-v2 found!


## 2) Set up the four main functions: get_chunks, get_prompt, ask_claude, and ask_claude_threaded

When we have a text to summarize, we'll use get_chunks to break it into the best size for preserving information.  get_prompt will build a prompt based on what we're trying to do, and finally ask_claude will send the prompt to Bedrock while ask_claude_threaded calls Bedrock in parallel on multiple threads.


In [22]:
def get_chunks(full_text, OVERLAP=True, DEBUG=False):
    '''
    This will take a text and return an array with sliced chunks of the text in optimal sizing for summarization.  Note that by default, this does include overlaping text in each chunk.
    Overlaping allows more cohesion between text, and should only be turned off when trying to count specific numbers and no duplicated text is a requirment.
    
    We could just drop text up to the maximum context window of our model, but that actually doesn't work very well.
    Part of the reason for this is because no matter the input length, the output length is about the same.
    For example, if you drop in a paragraph or 10 pages, you get about a paragraph in response.
    To mitigate this, we create chunks using the lesser of two values: 25% of the total token count or 2k tokens.
    We'll also overlap our chunks by about a paragraph of text or so, in order to provide continuity between chunks.
    (Logic taken from https://gist.github.com/Donavan/4fdb489a467efdc1faac0077a151407a)
    '''
    DEBUG = False #debugging at this level is usually not very helpful.
    
    #Following testing, it was found that chunks should be 2000 tokens, or 25% of the doc, whichever is shorter.
    #max chunk size in tokens
    chunk_length_tokens = 2000
    #chunk length may be shortened later for shorter docs.
    
    #a paragraph is about 200 words, which is about 260 tokens on average
    #we'll overlap our chunks by a paragraph to provide cohesion to the final summaries.
    overlap_tokens = 260
    if not OVERLAP: overlap_tokens = 0
    
    #anything this short doesn't need to be chunked further.
    min_chunk_length = 260 + overlap_tokens*2
    
    
    #grab basic info about the text to be chunked.
    char_count = len(full_text)
    word_count = len(full_text.split(" "))#rough estimate
    token_count = count_tokens(full_text)
    token_per_charater = token_count/char_count

    
    #don't chunk tiny texts
    if token_count <= min_chunk_length:
        if DEBUG: print("Text is too small to be chunked further")
        return [full_text]
    
    
    if DEBUG:
        print ("Chunk DEBUG mode is on, information about the text and chunking will be printed out.")
        print ("Estimated character count:",char_count)
        print ("Estimated word count:",word_count)
        print ("Estimated token count:",token_count)
        print ("Estimated tokens per character:",token_per_charater)

        print("Full text tokens: ",count_tokens(full_text))
        print("How many times bigger than max context window: ",round(count_tokens(full_text)/max_token_count,2))
    
    
    
    #if the text is shorter, use smaller chunks
    if (token_count/4<chunk_length_tokens):
        overlap_tokens = int((overlap_tokens/chunk_length_tokens)*int(token_count/4))
        chunk_length_tokens = int(token_count/4)
        
        if DEBUG: 
            print("Short doc detected:")
            print("New chunk length:",chunk_length_tokens)
            print("New overlap length:",overlap_tokens)
        
    #convert to charaters for easy slicing using our approximate tokens per character for this text.
    overlap_chars = int(overlap_tokens/token_per_charater)
    chunk_length_chars = int(chunk_length_tokens/token_per_charater)
    
    #itterate and create the chunks from the full text.
    chunks = []
    start_chunk = 0
    end_chunk = chunk_length_chars + overlap_chars
    
    last_chunk = False
    while not last_chunk:
        #the last chunk may not be the full length.
        if(end_chunk>=char_count):
            end_chunk=char_count
            last_chunk=True
        chunks.append(full_text[start_chunk:end_chunk])
        
        #move our slice location
        if start_chunk == 0:
            start_chunk += chunk_length_chars - overlap_chars
        else:
            start_chunk += chunk_length_chars
        
        end_chunk = start_chunk + chunk_length_chars + 2 * overlap_chars
        
    if DEBUG:print ("Created %s chunks."%len(chunks))
    return chunks

In [111]:
prompt_template = """\n\nHuman:  I am going to give you a text{{GUIDANCE_1}}.  This text is extracted from a larger document.  Here is the text:

<text>
{{TEXT}}
</text>
{{GUIDANCE_2}}
{{STYLE}}{{REQUEST}}{{FORMAT}}{{GUIDANCE_3}}
\nAssistant:  Here is what you asked for:
"""

merge_prompt_template = """\n\nHuman:  Here are a number of related summaries:

{{TEXT}}
Please merge these summaries into a highly detailed single summary in {{FORMAT}} format, preserving as much detail as possible, using less than 1000 tokens.
\nAssistant:  Here is what you asked for:
"""

#this is inserted into the prompt template above, in the {{GUIDANCE_2}} section.
guidance_tempate = """
Here is the additional guidance:
<guidance>
{{GUIDANCE}}
</guidance>
"""

#this prompt asks the LLM to be a newpaper reporter, extracting facts from a document to be used in a later report.  Good for summarizing factual sets of documents.
reporter_prompt = """\n\nHuman:  You are a newspaper reporter, collecting facts to be used in writing an article later.  Consider this source text:
<text>
{{TEXT}}
</text>
{{DOCS_DESCRIPTION}}  Please create a {{FORMAT}} of all the relevant facts from this text which will be useful in answering the question "{{GUIDANCE}}".  To make your list as clear as possible, do not use and pronouns or ambigious phrases.  For example, use a company's name rather than saying "the company" or they.
\nAssistant:  Here is the {{FORMAT}} of relevant facts:
"""

reporter_summary_prompt = """\n\nHuman:  You are a newspaper reporter, collecting facts to be used in writing an article later.  Consider these notes, each one derived from a different source text:
{{TEXT}}
Please create a {{FORMAT}} of all the relevant facts and trends from these notes which will be useful in answering the question "{{GUIDANCE}}"{{STYLE}}.  To make your list as clear as possible, do not use and pronouns or ambigious phrases.  For example, use a company's name rather than saying "the company" or "they".
\nAssistant:  Here is the list of relevant facts:

"""

reporter_final_prompt = """\n\nHuman:  You are a newspaper reporter, writing an article based on facts that were collected and summarized earlier.  Consider these summaries:
{{TEXT}}
Each summary is a collection of facts extracted from a number of source reports.  Each source report was written by an AWS team talking about their interactions with their individual customer.  Please create a {{FORMAT}} of all the relevant trends and details from these summaries which will be useful in answering the question "{{GUIDANCE}}".
\nAssistant:  Here is the narrative:

"""

def get_prompt(text,prompt_type,format_type, manual_guidance, style_guide, docs_description=""):
    '''
    text should be a single string of the raw text to be sent to the gen ai model.
    prompt_type must be "summary" or "interrogate" or "answers"
            -summary means summarize the text
            -interrogate means look at the text and ask questions about what is missing
            -answers means looking at the test, provide only details that may help answer the questions according to the Guidance.
            -merge_answers takes a summary as text, and merges in the facts in the guidance section
            -merge_summaries takes 2 or more summaries and merges them together.  The summaries to be merged must be in list format for best results.
            -reporter - like a new reporter, extract details that help answer the guidance questions
            -reporter_summary - like a news reporter looking at a bunch of notes, create a list summary.  Intended as an intermediate step. 
            reporter_final - generative a narrative based on the reporter_summary outputs.
    format_type must be "narrative" or "list"
    manual_guidance Extra instructions to guide the process, usually from the user.
    style_guide TBD
    
    Note that merge_summaries is handled differntly than all other options because it iteratively adds in multiple texts.
    '''
    
    #answers mode is a bit different, so handle that first.
    if prompt_type == "answers":
        format_type = "in list format, using less than 1000 tokens.  "
        prompt_type = "Please provide a list of any facts from the text that could be relevant to answering the questions from the guidance section "
        guidance_1 = " and some guidance"
        guidance_2 = guidance_tempate.replace("{{GUIDANCE}}",manual_guidance)
        guidance_3 = "You should ignore any questions that can not be answered by this text."
    elif prompt_type == "reporter":
        return reporter_prompt.replace("{{TEXT}}",text).replace("{{FORMAT}}",format_type).replace("{{GUIDANCE}}",manual_guidance).replace("{{DOCS_DESCRIPTION}}",docs_description)
    elif prompt_type == "reporter_summary":
        summaries_text = ""
        for x,summary in enumerate(text):
            summaries_text += "<note_%s>\n%s</note_%s>\n"%(x+1,summary,x+1)
        final_prompt = reporter_summary_prompt.replace("{{TEXT}}",summaries_text).replace("{{FORMAT}}",format_type).replace("{{GUIDANCE}}",manual_guidance).replace("{{STYLE}}",style_guide)
        return final_prompt
    elif prompt_type == "reporter_final":
        summaries_text = ""
        for x,summary in enumerate(text):
            summaries_text += "<summary_%s>\n%s</summary_%s>\n"%(x+1,summary,x+1)
        final_prompt = reporter_final_prompt.replace("{{TEXT}}",summaries_text).replace("{{FORMAT}}",format_type).replace("{{GUIDANCE}}",manual_guidance)
        return final_prompt
    elif prompt_type == "merge_summaries":
        summaries_text = ""
        for x,summary in enumerate(text):
            summaries_text += "<summary_%s>\n%s</summary_%s>\n"%(x+1,summary,x+1)
        final_prompt = merge_prompt_template.replace("{{TEXT}}",summaries_text).replace("{{FORMAT}}",format_type)
        return final_prompt
        
    elif prompt_type == "merge_answers":
        prompt_type = "The text is a good summary which may lack a few details.  However, the additional information found in the guidance section can be used to make the summary even better.  Starting with the text, please use the details in the guidance section to make the text more detailed.  The new summary shoud use less than 1000 tokens.  "
        format_type = ""
        guidance_1 = " and some guidance"
        guidance_2 = guidance_tempate.replace("{{GUIDANCE}}",manual_guidance)
        guidance_3 = "You should ignore any comments in the guidance section indicating that answers could not be found."
    else:
        #Based on the options passed in, grab the correct text to eventually use to build the prompt.
        #select the correct type of output format desired, list or summary.  Note that list for interrogate prompts is empty because the request for list is built into that prompt.
        if prompt_type == "interrogate" and format_type != "list":
            raise ValueError("Only list format is supported for interrogate prompts.")
        if format_type == "list":
            if prompt_type == "interrogate":
                format_type = ""#already in the prompt so no format needed.
            else:
                format_type = "in list format, using less than 1000 tokens."
        elif format_type == "narrative":
            format_type = "in narrative format, using less than 1000 tokens."
        else:
            raise ValueError("format_type must be 'narrative' or 'list'.")

        #select the correct prompt type language
        if prompt_type == "summary":
            prompt_type = "Please provide a highly detailed summary of this text "
        elif prompt_type == "interrogate":
            prompt_type = "This text is a summary that lacks detail.  Please provide a list of the top 10 most important questions about this text that can not be answered by the text."
        else:
            raise ValueError("prompt_type must be 'summary' or 'interrogate'.")

        if manual_guidance == "":
            guidance_1 = ""
            guidance_2 = ""
            guidance_3 = ""
        else:
            guidance_1 = " and some guidance"
            guidance_2 = guidance_tempate.replace("{{GUIDANCE}}",manual_guidance)
            guidance_3 = "  As much as possible, also follow the guidance from the guidance section above.  You should ignore guidance that does not seem relevant to this text."
        
    #TBD
    style_guide = ""
    #print (prompt_template.replace("{{GUIDANCE_1}}",guidance_1).replace("{{GUIDANCE_2}}",guidance_2).replace("{{GUIDANCE_3}}",guidance_3).replace("{{STYLE}}",style_guide).replace("{{REQUEST}}",prompt_type).replace("{{FORMAT}}",format_type))
    final_prompt = prompt_template.replace("{{TEXT}}",text).replace("{{GUIDANCE_1}}",guidance_1).replace("{{GUIDANCE_2}}",guidance_2).replace("{{GUIDANCE_3}}",guidance_3).replace("{{STYLE}}",style_guide).replace("{{REQUEST}}",prompt_type).replace("{{FORMAT}}",format_type)
    return final_prompt

In [25]:
#Save our cache of calls to Claude
#this speeds things up when testing, because we're often making the same calls to Claude over and over.
#where the pickle is may change depending on where this file is run.
claude_cache_pickel = "claude_cache.pkl"
    
def save_calls(claude_cache):
    with open(claude_cache_pickel, 'wb') as file:
        pickle.dump(claude_cache,file)
#load our cached calls to Claude
def load_calls():
    with open(claude_cache_pickel, 'rb') as file:
        return pickle.load(file)
def clear_cache():
    claude_cache = {}
    save_calls()
#a cache of recent requests, to speed up itteration while testing
claude_cache = {}

if not os.path.exists(claude_cache_pickel):
    print ("Creating new, empty cache of Claude calls.")
    save_calls(claude_cache)

if CACHE_RESPONCES:
    claude_cache = load_calls()

Creating new, empty cache of Claude calls.


In [51]:
MAX_ATTEMPTS = 30 #how many times to retry if Claude is not working.
def ask_claude(prompt_text, DEBUG=False):
    '''
    Send a prompt to Bedrock, and return the response.  Debug is used to see exactly what is being sent to and from Bedrock.
    TODO:  Add error checking and retry on hitting the throttling limit.
    '''
    #usually, the prompt will have "human" and "assistant" tags already.  These are required, so if they are not there, add them in.
    if not "Assistant:" in prompt_text:
        prompt_text = "\n\nHuman:"+prompt_text+"\n\Assistant: "
        
    promt_json = {
        "prompt": prompt_text,
        "max_tokens_to_sample": 3000,
        "temperature": 0.7,
        "top_k": 250,
        "top_p": 0.7,
        "stop_sequences": ["\n\nHuman:"]
    }
    body = json.dumps(promt_json)
    
    #returned cashed results, if any
    if body in claude_cache:
        return claude_cache[body]
    
    if DEBUG: print("sending:",prompt_text)
    modelId = 'anthropic.claude-v2'
    accept = 'application/json'
    contentType = 'application/json'
    
    start_time = time.time()
    attempt = 1
    while True:
        try:
            query_start_time = time.time()
            response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
            response_body = json.loads(response.get('body').read())

            raw_results = response_body.get("completion").strip()

            #strip out HTML tags that Claude sometimes adds, such as <text>
            results = re.sub('<[^<]+?>', '', raw_results)
            request_time = round(time.time()-start_time,2)
            if DEBUG:
                print("Recieved:",results)
                print("request time (sec):",request_time)
            total_tokens = count_tokens(prompt_text+raw_results)
            output_tokens = count_tokens(raw_results)
            tokens_per_sec = round(total_tokens/request_time,2)
            break
        except Exception as e:
            print("Error with calling Bedrock: "+str(e))
            attempt+=1
            if attempt>MAX_ATTEMPTS:
                print("Max attempts reached!")
                results = str(e)
                request_time = -1
                total_tokens = -1
                output_tokens = -1
                tokens_per_sec = -1
                break
            else:#retry in 10 seconds
                time.sleep(10)
    #store in cache only if it was not an error:
    if request_time>0:
        claude_cache[body] = (prompt_text,results,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time)
    
    return(prompt_text,results,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time)

In [49]:
from queue import Queue
from threading import Thread

# Threaded function for queue processing.
def thread_request(q, result):
    while not q.empty():
        work = q.get()                      #fetch new work from the Queue
        thread_start_time = time.time()
        try:
            data = ask_claude(work[1])
            result[work[0]] = data          #Store data back at correct index
        except Exception as e:
            error_time = time.time()
            print('Error with prompt!',str(e))
            result[work[0]] = (work[1],str(e),count_tokens(work[1]),0,round(error_time-thread_start_time,2),0,thread_start_time)
        #signal to the queue that task has been processed
        q.task_done()
    return True

def ask_claude_threaded(prompts,DEBUG=False):
    '''
    Call ask_claude, but multi-threaded.
    Returns a dict of the prompts and responces.
    '''
    q = Queue(maxsize=0)
    num_theads = min(50, len(prompts))
    
    #Populating Queue with tasks
    results = [{} for x in prompts];
    #load up the queue with the promts to fetch and the index for each job (as a tuple):
    for i in range(len(prompts)):
        #need the index and the url in each queue item.
        q.put((i,prompts[i]))
        
    #Starting worker threads on queue processing
    for i in range(num_theads):
        #print('Starting thread ', i)
        worker = Thread(target=thread_request, args=(q,results))
        worker.setDaemon(True)    #setting threads as "daemon" allows main program to 
                                  #exit eventually even if these dont finish 
                                  #correctly.
        worker.start()

    #now we wait until the queue has been processed
    q.join()

    if DEBUG:print('All tasks completed.')
    return results

In [134]:
#test if a singe Claude call is working
#print("Testing!  Is Claude working? "+ask_claude("Is Claude working?")[1])

Testing!  Is Claude working? I'm afraid I don't have enough information to know if someone named Claude is working or not. As an AI assistant without personal knowledge of specific people, I can't make assumptions about whether a particular person is currently working or not. I'd need more context to determine that.


In [34]:
#test if our threaded Claude calls are working
#print(ask_claude_threaded(["Please say the number one.","Please say the number two."]))

/tmp/ipykernel_2825/662785133.py:38: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True)    #setting threads as "daemon" allows main program to


All tasks completed.
[('\n\nHuman:Please say the number one.\n\\Assistant: ', 'One.', 16, 2, 0.55, 29.09, 1698343933.9464455), ('\n\nHuman:Please say the number two.\n\\Assistant: ', 'Two.', 16, 2, 2.65, 6.04, 1698343936.056085)]


##   3) Set up the main summarizing itterative functions, single and multi doc, using the four helper functions from part (2).

In [58]:
def generate_summary_from_chunks(chunks, prompt_options,DEBUG=False, chunks_already_summarized=False):
    """
    This function itterates through a list of chunks, summarizes them, then merges those summaries together into one.
    chunks_already_summarized is used when the chunks passed in are chunks resulting from summerizing docs.
    If the chunks are taken from a source document directly, chunks_already_summarized should be set to False.
    """
    partial_summaries = {}
    if not chunks_already_summarized:#chunks are from a source doc, so summarize them.
        partial_summaries_prompts = []
        partial_summaries_prompt2chunk = {}
        for x,chunk in enumerate(chunks):
            #if DEBUG: print ("Working on chunk",x+1,end = '')
            start_chunk_time = time.time()
            #note that partial summaries are always done in list format to maximize information captured.
            custom_prompt = get_prompt(chunk,prompt_options['prompt_type'],'list', prompt_options['manual_guidance'], prompt_options['style_guide'])
            #partial_summaries[chunk] = ask_claude(custom_prompt,DEBUG=False)
            partial_summaries_prompts.append(custom_prompt)
            partial_summaries_prompt2chunk[custom_prompt]=chunk
        
        partial_summaries_results = ask_claude_threaded(partial_summaries_prompts)
        for prompt_text,results,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time in partial_summaries_results:
            partial_summaries[partial_summaries_prompt2chunk[prompt_text]] = results

        if DEBUG: 
            print ("Partial summary chunks done!")
            print ("Creating joint summary...")
    else:
        for chunk in chunks:
            partial_summaries[chunk] = chunk
        if DEBUG: 
            print ("Summarized chunks detected!")
            print ("Creating joint summary...")
            
    summaries_list = []
    summaries_list_token_count = 0
    for chunk in chunks:
        summaries_list.append(partial_summaries[chunk]) 
        summaries_list_token_count+=count_tokens(partial_summaries[chunk])
        
    if DEBUG: print("Chunk summaries token count:",summaries_list_token_count)
    
    #check to see if the joint summary is too long.  If it is, recursivly itterate down.
    #we do this, rather than chunking again, so that summaries are not split.
    #it needs to be under 3000 tokens in order to be helpful to the summary (4000 is an expiremental number and may need to be adjusted.)
    #this may be higher than the 2000 used for text originally, because this data is in list format.
    recombine_token_target = 3000
    #summaries_list_token_count = recombine_token_target+1 #set this to target+1 so that we do at least one recombonation for shorter documents.
    while summaries_list_token_count>recombine_token_target:
        if DEBUG: print("Starting reduction loop to merge chunks.  Total token count is %s"%summaries_list_token_count)
        new_summaries_list = []
        summaries_list_token_count = 0
        temp_summary_group = []
        temp_summary_group_token_length = 0
        for summary in summaries_list:
            if temp_summary_group_token_length + count_tokens(summary) > recombine_token_target:
                #the next summary added would push us over the edge, so summarize the current list, and then add it.
                #note that partial summaries are always done in list format to maximize information captured.
                if DEBUG: print("Reducing %s partial summaries into one..."%(len(temp_summary_group)))
                custom_prompt = get_prompt(temp_summary_group,"merge_summaries","list", prompt_options['manual_guidance'], prompt_options['style_guide'])
                temp_summary = ask_claude(custom_prompt,DEBUG=False)[1]
                new_summaries_list.append(temp_summary)
                summaries_list_token_count+= count_tokens(temp_summary)
                temp_summary_group = []
                temp_summary_group_token_length = 0
            
            temp_summary_group.append(summary)
            temp_summary_group_token_length+= count_tokens(summary)
        
        #summarize whever extra summaries are still in the temp list
        if len(temp_summary_group)>1:
            if DEBUG: print("Starting final reduction of %s partial summaries into one..."%(len(temp_summary_group)))
            custom_prompt = get_prompt(temp_summary_group,"merge_summaries","list", prompt_options['manual_guidance'], prompt_options['style_guide'])
            temp_summary = ask_claude(custom_prompt,DEBUG=False)[1]
            new_summaries_list.append(temp_summary)
            summaries_list_token_count+= count_tokens(temp_summary)
        elif len(temp_summary_group)==1:
            if DEBUG: print("Tacking on an extra partial summary")
            new_summaries_list.append(temp_summary_group[0])
            summaries_list_token_count+= count_tokens(temp_summary_group[0])
            
        summaries_list = new_summaries_list
        
    if DEBUG: print ("Final merge of summary chunks, merging %s summaries."%(len(summaries_list)))
    custom_prompt = get_prompt(summaries_list,"merge_summaries",prompt_options['format_type'], prompt_options['manual_guidance'], prompt_options['style_guide'])
    full_summary = ask_claude(custom_prompt,DEBUG=False)[1]
    #full_summary_prompt = get_prompt("/n".join(summaries_list),prompt_options['prompt_type'],prompt_options['format_type'], prompt_options['manual_guidance'], prompt_options['style_guide'])
    #full_summary = ask_claude(full_summary_prompt,DEBUG=False)
    
    return full_summary

In [44]:
def generate_single_doc_summary(full_text, prompt_options,AUTO_REFINE=True, DEBUG=False,ALREADY_CHUNKED_AND_SUMMED=False):
    """
    This function uses the three helper functions, as well as the generate_summary_from_chunks above, to iteratively generate high quality summaries.
    AUTO_REFINE, if true, has the LLM generate a list of questions, and then recursivly calls this function with those questions for guidance.
    ALREADY_CHUNKED_AND_SUMMED, if true, means that this is being called using a list of summarized documents which should not be chunked or summarized further.
    """
    #first break this document into chunks
    chunks = []        
    
    if ALREADY_CHUNKED_AND_SUMMED:
        chunks = full_text
    else:
        chunks = get_chunks(full_text,DEBUG=DEBUG)
        
    if DEBUG:
        if prompt_options['prompt_type'] == "answers":
            print ("Generating answers using %s chunks."%(len(chunks)))
        else:
            print ("Generating a new combined summary for %s chunks."%(len(chunks)))
        if ALREADY_CHUNKED_AND_SUMMED:
            print ("Input has already been chunked and summarized, skipping initial chunking.")
        
            
    first_summary = generate_summary_from_chunks(chunks,prompt_options,DEBUG=DEBUG, chunks_already_summarized=ALREADY_CHUNKED_AND_SUMMED)
    
    if DEBUG and AUTO_REFINE: 
        print ("First summary:")
        print (first_summary)
        
    if AUTO_REFINE: 
        if DEBUG: print ("Asking the LLM to find weaknesses in this summary...")
        #now that we have a rough summary, let's grab some questions about it.
        questions_prompt = get_prompt(first_summary,"interrogate","list", "", "")
        questions_list = ask_claude(questions_prompt,DEBUG=False)[1]

        if DEBUG: 
            print ("Questions from the LLM:")
            print (questions_list)
            
        original_guidance = prompt_options['manual_guidance']
        original_prompt_type = prompt_options['prompt_type']
        prompt_options['manual_guidance'] = prompt_options['manual_guidance'] + questions_list
        prompt_options['prompt_type'] = "answers"
        add_details = generate_single_doc_summary(full_text, prompt_options,AUTO_REFINE=False, DEBUG=DEBUG, ALREADY_CHUNKED_AND_SUMMED=ALREADY_CHUNKED_AND_SUMMED)
        if DEBUG: 
            print("Additional Details:")
            print (add_details)
            print("Merging details into original summary...")
        
        prompt_options['manual_guidance'] = original_guidance + add_details
        prompt_options['prompt_type'] = "merge_answers"
        custom_prompt = get_prompt(first_summary,prompt_options['prompt_type'],prompt_options['format_type'], prompt_options['manual_guidance'], prompt_options['style_guide'])
        final_summary = ask_claude(custom_prompt,DEBUG=False)[1]
        
        #return this back to the original to prevent weird errors between calls of this function.
        prompt_options['manual_guidance'] = original_guidance
        prompt_options['prompt_type'] = original_prompt_type
        return final_summary
    
    else:
        return first_summary

In [69]:
def grab_set_chunks(lst, n):
    """Yield successive n-sized chunks from lst.
    This is a helper function for the multidoc summarization function.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [112]:
def generate_multiple_docs_summary(docs, questions, docs_description, DEBUG=False):
    """
    This function uses the three helper functions to read the documents passed in, and create a summary answer for each question passed in.
    If the documents are longer than two pages or so, it is reccoemended that you first summaize each document.
    docs_description is a single sentance describing what the documents are such as "The texts are a collection of product reviews for a Pickle Ball paddle."
    """
    #get answers from each doc for each question.
    answers = {}
    prompt2quetion_doc = {}
    prompts = []
    max_docs_to_scan = 500
    
    #build the queries to be passed into Bedrock
    for question in questions:
        for x,doc in enumerate(docs):
            if x>max_docs_to_scan:break#limit for testing
            
            #print ("Asking the LLM to find extract answers from this doc:",doc)
            questions_prompt = get_prompt(docs[doc],"reporter","list", question, "",docs_description)
            prompt2quetion_doc[questions_prompt] = (question,doc) 
            prompts.append(questions_prompt)
        
    if DEBUG:print("Starting %s worker threads."%len(prompts))
    prompts_answers = ask_claude_threaded(prompts,DEBUG=False)
    save_calls(claude_cache)
    
    for question in questions:
        answers[question] = []    
    
    for prompt,answer,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time in prompts_answers:
        question,doc = prompt2quetion_doc[prompt]
        answers[question].append(answer)
        
    
    current_answer_count = len(docs)
    if DEBUG: print("All documents have been read.  Reducing answers into the final summary...")
    #reduce this down to 5 or less docs for the final summary by combining the individual answers.
    while current_answer_count > 5:
        #summarize the answers
        prompts = []
        prompts2question = {}
        
        max_docs_to_scan = max(min(current_answer_count,8),3)
        if DEBUG: print("Combining %s chunks.  (Currently there are %s answers to each question.)"%(max_docs_to_scan,current_answer_count))
        for question in questions:
            #print ("Asking the LLM to summarize answers for this question:",question)
            #You want chunks of roughly 2K tokens
            for partial_chunks in grab_set_chunks(answers[question],max_docs_to_scan):
                questions_prompt = get_prompt(partial_chunks,"reporter_summary","list", question, " in less than 1000 tokens")
                prompts.append(questions_prompt)
                prompts2question[questions_prompt] = question
        
        if DEBUG:print("Starting %s worker threads."%len(prompts))
        prompts_answers = ask_claude_threaded(prompts,DEBUG=False)
        save_calls(claude_cache)
        
        for question in questions:
            answers[question] = []    
        for prompt,answer,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time in prompts_answers:
            answers[prompts2question[prompt]].append(answer)        

        current_answer_count = len(answers[questions[0]])
        
    if DEBUG: print("Creating the final summary for each question.")
    #write the final article:
    prompts = []
    prompts2question = {}
    for question in questions:
        #print ("Asking the LLM to finalize the answer for this question:",question)
        questions_prompt = get_prompt(answers[question],"reporter_final","narrative", question, "")
        prompts.append(questions_prompt)
        prompts2question[questions_prompt] = question

    if DEBUG:print("Starting %s worker threads."%len(prompts))
    prompts_answers = ask_claude_threaded(prompts,DEBUG=False)
    save_calls(claude_cache)
    
    answers = {}
    for prompt,answer,total_tokens,output_tokens,request_time,tokens_per_sec,query_start_time in prompts_answers:
        answers[prompts2question[prompt]] = answer
    return answers
    

## 4) Explore using the main summarizing function for maximum summary quality.
For this notebook, we will load sample text from pickles created in the Data Collection and Cleaning notebook included in this repo.
If you want to use your own data, check out the Data Collection and Cleaning for examples of how to format the pickles used here.

### Example using one long document

In [136]:
#change to True to run examples.  It's false by default so that this notebook can be used as a resouce elsewhere.
RUN_EXAMPLES = False
if __name__ == '__main__':
    RUN_EXAMPLES = True

In [137]:
if RUN_EXAMPLES:
    #file locations for pickels of text.  These are a single string containing the whole text.
    #shore, medium, and long texts are provided as exampels.
    text_to_open_short = 'sample texts/hills.pkl'  #2-3 page story, Hills like White Elephants
    text_to_open_mid = 'sample texts/algernon.pkl'  #short story, Flowers for Algernon
    text_to_open_long = 'sample texts/frankenstien.pkl' #short novel, Frankenstine
    text_to_open_short_factual = 'sample texts/elvis.pkl'  #longest wikipedia article, Elvis.

    with open(text_to_open_mid, 'rb') as file:
        full_text = pickle.load(file)
    print (len(full_text))

140397


In [138]:
#%%time
if RUN_EXAMPLES:
    prompt_options = {}
    prompt_options['prompt_type'] = "summary"
    prompt_options['format_type'] = "narrative"
    #prompt_options['manual_guidance'] = "Assume you are an AWS Sales Executive.  Please include two sections in the summary, one for highlights and positive reactions, and one for challenges and negative reactions."
    prompt_options['manual_guidance'] = ""
    prompt_options['style_guide'] = ""

    revised_summary = generate_single_doc_summary(full_text, prompt_options, AUTO_REFINE=True, DEBUG=True)
    print ("Final Summary:")
    print (revised_summary)
    #put call history in pickle
    save_calls(claude_cache)

Generating a new combined summary for 18 chunks.
Working on chunk 1Working on chunk 2Working on chunk 3Working on chunk 4Working on chunk 5Working on chunk 6Working on chunk 7Working on chunk 8Working on chunk 9Working on chunk 10Working on chunk 11Working on chunk 12Working on chunk 13Working on chunk 14Working on chunk 15Working on chunk 16Working on chunk 17Working on chunk 18All tasks completed.
Partial summary chunks done!
Creating joint summary...
Chunk summaries token count: 4619
Starting reduction loop to merge chunks.  Total token count is 4619
Reducing 11 partial summaries into one...
Starting final reduction of 7 partial summaries into one...
Final merge of summary chunks, merging 2 summaries.
First summary:
Charlie Gordon is a man with an intellectual disability who undergoes an experimental surgery to increase his intelligence. He starts writing progress reports to document the changes. Charlie goes to the Beekman College Center for Retarded Adults to learn reading, writin

/tmp/ipykernel_2825/506474523.py:39: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True)    #setting threads as "daemon" allows main program to


Generating answers using 18 chunks.
Working on chunk 1Working on chunk 2Working on chunk 3Working on chunk 4Working on chunk 5Working on chunk 6Working on chunk 7Working on chunk 8Working on chunk 9Working on chunk 10Working on chunk 11Working on chunk 12Working on chunk 13Working on chunk 14Working on chunk 15Working on chunk 16Working on chunk 17Working on chunk 18All tasks completed.
Partial summary chunks done!
Creating joint summary...
Chunk summaries token count: 3967
Starting reduction loop to merge chunks.  Total token count is 3967
Reducing 13 partial summaries into one...
Starting final reduction of 5 partial summaries into one...
Final merge of summary chunks, merging 2 summaries.
Additional Details:
Charlie underwent an experimental surgery and injections to increase his intelligence, though the specifics of the procedure were not detailed. Psychologists administered various tests to evaluate Charlie's progress, including inkblots, mazes, and puzzles, but did not provide th

### Multi-doc summarization example

In [122]:
RUN_EXAMPLES = False
if __name__ == '__main__':
    RUN_EXAMPLES = True

In [72]:
#quick doc summary grab
if RUN_EXAMPLES:
    text_to_open_doc_group = 'sample texts/docs.pkl'
    with open(text_to_open_doc_group, 'rb') as file:
        docs = pickle.load(file)
    print (len(docs))

10


In [73]:
questions = ["What are the highlights?",
"What are the challenges?"]


In [113]:
#%%time
if RUN_EXAMPLES:
    doc_description = "The texts are a collection of product reviews for a Pickle Ball paddle."
    answers = generate_multiple_docs_summary(docs, questions, doc_description, DEBUG=True)

    for question in answers:
        print ()
        print (question)
        print ()
        print (answers[question].replace("\n\n","\n"))


Starting 20 worker threads.


/tmp/ipykernel_2825/506474523.py:39: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True)    #setting threads as "daemon" allows main program to


All tasks completed.
All documents have been read.  Reducing answers into the final summary...
Combining 8 chunks.  (Currently there are 10 answers to each question.)
Starting 4 worker threads.
All tasks completed.
Creating the final summary for each question.
Starting 2 worker threads.
All tasks completed.

What are the highlights?

The Panel Sound pickleball set seems to provide good value and convenience for most customers, though some have had issues with durability. The set includes lightweight fiberglass paddles, indoor and outdoor balls, a carrying case, and cooling towels. Reviewers found the paddles easy to use and maneuverable, though they may not suit all players' preferences. One reviewer was new to pickleball and found the set a good way to try the sport. Some found the cooling towels small. 
There were a couple cases of Panel Sound paddles coming loose or breaking during use after several months. One customer could not get a refund or replacement due to the 30 day return 

#### Save Output

In [117]:
if RUN_EXAMPLES:
    for question in questions:
        with open("results/"+question+".txt","w") as fout:
            fout.write(answers[question])

## Enable a Command Line interface, for interactive testing from CLI

To use this, first convert the notebook to a python script, then run that from the CLI.

jupyter nbconvert --to python summarize.ipynb 

In [123]:
enable_CLI = False

In [127]:
if enable_CLI:
    print ("Loading data from pickle.")
    text_to_open_doc_group = 'example.pkl'
    with open(text_to_open_doc_group, 'rb') as file:
        docs = pickle.load(file)
    print("%s docs loaded."%len(docs))
    
    print ("Please enter a single sentence description of what the docs are.  For example \"These documents are reviews from Amazon.com\"")
    doc_description = input()
    while True:
        print ("\nWhat would you like to ask? (type quit to end the program.)\n")
        questions = [input()]
        
        if questions[0] in ["quit","q"]:
            break
        
        answers = generate_multiple_docs_summary(docs, questions, doc_description, DEBUG=True)
        
        for question in answers:
            print ()
            print (question)
            print ()
            print (answers[question].replace("\n\n","\n"))
            
        
        

Loading data from pickle.
8 docs loaded.
Please enter a single sentence description of what the docs are.  For example "These documents are reviews from Amazon.com"


 The documents are a sales summaries for AWS.



What would you like to ask? (type quit to end the program.)



 quit
